In [62]:
%reset

In [63]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RationalQuadratic, Matern, WhiteKernel, RBF
from sklearn.gaussian_process.kernels import Sum
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from scipy import stats

import numpy as np
import pandas as pd
import optuna
import warnings

In [64]:
#Set this value to true if hyperparameter tuning is complete and the test set should be loaded and predicted on
OUTPUT_TEST = False

In [65]:
#Load the training and validation datasets
X_train = pd.read_csv("../data/cleaned/training.csv")
y_train = pd.read_csv("../data/cleaned/training_labels.csv")
X_val = pd.read_csv("../data/cleaned/validation.csv")
y_val = pd.read_csv("../data/cleaned/validation_labels.csv")

In [66]:
#Some columns headers contain '[' or ']' which are not compatable with sklearn. They are change to '(' and ')' respectively.
columns = X_train.columns
for col in columns:
    if '[' in col or ']' in col:
        old_name = col
        col = col.replace('[', '(')
        col = col.replace(']', ')')
        
        X_train = X_train.rename(columns={old_name:col})
        X_val = X_val.rename(columns={old_name:col})

In [67]:
#Splitting of the training set into a vedrification and training set with a 90/10 split. This verification set is used for optuna hyperparameter tuning.
X_train, X_verif, y_train, y_verif = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [68]:
#Reset the indicies after splitting the dataset
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_verif = X_verif.reset_index(drop=True)
y_verif = y_verif.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)

In [69]:
#Defining the guassian process search space for Optuna.
def define_kernel(trial):
    kernels = []
    n_kernels = trial.suggest_int('n_kernels', 1, 3) #Number of simple kernels used to create the final kernel
    for i in range(n_kernels):
        kernel_type = trial.suggest_categorical(f'kernel_type_{i}', ["Matern", "RationalQuadratic"]) #Select a type of simple kernel

        #Depending on the kernel type selected, certain metrics need to specificed, each of those metrics is selected below by optuna as the tuning takes place
        if kernel_type == 'RationalQuadratic':
            quad_params = {
                'length_scale': trial.suggest_float(f'RationalQuadratic_{i}_length_scale', 1e-1, 1e5),
                'alpha': trial.suggest_float(f'RationalQuadratic_{i}_alpha', 1e-1, 1e5)
            }   
            kernel = RationalQuadratic(length_scale=quad_params['length_scale'], alpha=quad_params['alpha'], length_scale_bounds=(1e-8,1e8))
        elif kernel_type == 'Matern':
            matern_params = {
                'length_scale': trial.suggest_float(f'Matern_{i}_length_scale', 1e-1, 1e5),
                'nu': trial.suggest_float(f'Matern_{i}_nu', 0.5, 5)
            }
            kernel = Matern(length_scale=matern_params['length_scale'], nu=matern_params['nu'], length_scale_bounds=(1e-8,1e8))
        else:
            print("WRONG KERNEL NAME FOR:", kernel_type)
            TypeError
        kernels.append(kernel)

    white_noise_params = {
                'noise_level': trial.suggest_float(f'WhiteKernel_{n_kernels}_noise_level', 1e-5, 1e5),
            }
    #kernel = WhiteKernel(noise_level=white_noise_params['noise_level'])
    #kernels.append(kernel)
    if n_kernels == 1:
        combined_kernel = kernels[0]
    else:
        combined_kernel = Sum(kernels[0], kernels[1])
        for j in range(1, n_kernels-1):
            combined_kernel = Sum(combined_kernel, kernels[j+1])
    return combined_kernel

In [70]:
def objective(trial):
    """Define the objective function"""
    kernel = define_kernel(trial)
    params = {
        'alpha': trial.suggest_float('alpha', 1e-3, 1e3, log=True),
        'n_restarts_optimizer': trial.suggest_int('n_restarts_optimizer', 0, 10),
    }

    params["kernel"] = kernel
    print(params)
    # Fit the model
    optuna_model = GaussianProcessRegressor(**params)
    
    batch_size = 500

    # Take a random sample of the DataFrame
    X_train_sampled = X_train.sample(n=batch_size)

    # Access the indexes of the sampled rows
    sampled_indexes = X_train_sampled.index
    y_train_sampled = y_train.loc[sampled_indexes]

    optuna_model.fit(X_train_sampled, y_train_sampled)
    
    # Make predictions
    verif_pred = optuna_model.predict(X_verif)
    verif_loss = mean_absolute_percentage_error(y_verif,verif_pred)*100
    verif_error = mean_squared_error(y_verif,verif_pred,squared=False)

    # Evaluate predictions
    error = verif_loss + verif_error
    
    return error

In [71]:
warnings.filterwarnings("ignore", category=RuntimeWarning)
study = optuna.create_study(pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2023-12-19 17:22:42,257] A new study created in memory with name: no-name-67ce1b1a-c429-4e04-ba6b-7562cf0fa2ac


{'alpha': 0.4322961695854962, 'n_restarts_optimizer': 10, 'kernel': RationalQuadratic(alpha=2.77e+04, length_scale=3.85e+04) + Matern(length_scale=8.92e+04, nu=1.73) + Matern(length_scale=4.56e+03, nu=4.55)}


c:\Users\nickr\anaconda3\envs\ml_project\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\nickr\anaconda3\envs\ml_project\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__length_scale is close to the specified lower bound 1e-08. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
[I 2023-12-19 17:26:33,190] Trial 0 finished with value: 48.51528888534959 and parameters: {'n_kernels': 3, 'kernel_type_0': 'RationalQuadratic', 'RationalQuadratic_0_length_scale': 38541.8416897705, 'RationalQuadratic_0_alpha': 27689.71642443232, 'kernel_type_1': 'Matern', 'Matern_1_length_scale': 89153.0

{'alpha': 1.2212138429940311, 'n_restarts_optimizer': 1, 'kernel': Matern(length_scale=7.86e+04, nu=0.848) + RationalQuadratic(alpha=7.77e+04, length_scale=8.73e+04)}


c:\Users\nickr\anaconda3\envs\ml_project\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-08. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
[I 2023-12-19 17:27:00,680] Trial 1 finished with value: 49.66094240423003 and parameters: {'n_kernels': 2, 'kernel_type_0': 'Matern', 'Matern_0_length_scale': 78623.28077857281, 'Matern_0_nu': 0.8476520284471216, 'kernel_type_1': 'RationalQuadratic', 'RationalQuadratic_1_length_scale': 87259.83313914941, 'RationalQuadratic_1_alpha': 77720.5492153229, 'WhiteKernel_2_noise_level': 39932.49011801804, 'alpha': 1.2212138429940311, 'n_restarts_optimizer': 1}. Best is trial 0 with value: 48.51528888534959.


...

c:\Users\nickr\anaconda3\envs\ml_project\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-08. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
[I 2023-12-19 17:53:04,037] Trial 16 finished with value: 49.91763999747879 and parameters: {'n_kernels': 3, 'kernel_type_0': 'Matern', 'Matern_0_length_scale': 72455.101666054, 'Matern_0_nu': 3.6446069027724644, 'kernel_type_1': 'Matern', 'Matern_1_length_scale': 25040.77712417939, 'Matern_1_nu': 4.109579879220222, 'kernel_type_2': 'RationalQuadratic', 'RationalQuadratic_2_length_scale': 99570.08716297068, 'RationalQuadratic_2_alpha': 804.1040980528269, 'WhiteKernel_3_noise_level': 75925.14252745925, 'alpha': 0.03544444647030521, 'n_restarts_optimizer': 10}. Best is trial 0 with value: 48.51528888534959.


{'alpha': 0.0012661989972729496, 'n_restarts_optimizer': 8, 'kernel': RationalQuadratic(alpha=4.13e+04, length_scale=2.71e+03) + Matern(length_scale=4.36e+04, nu=3.3) + Matern(length_scale=9.71e+04, nu=2.77)}


c:\Users\nickr\anaconda3\envs\ml_project\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__length_scale is close to the specified lower bound 1e-08. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
[I 2023-12-19 17:55:04,379] Trial 17 finished with value: 49.99931819406822 and parameters: {'n_kernels': 3, 'kernel_type_0': 'RationalQuadratic', 'RationalQuadratic_0_length_scale': 2706.879296918072, 'RationalQuadratic_0_alpha': 41296.41221101981, 'kernel_type_1': 'Matern', 'Matern_1_length_scale': 43576.496980144766, 'Matern_1_nu': 3.3031630250424016, 'kernel_type_2': 'Matern', 'Matern_2_length_scale': 97104.08995075297, 'Matern_2_nu': 2.770954131503216, 'WhiteKernel_3_noise_level': 23106.93990919988, 'alpha': 0.0012661989972729496, 'n_restarts_optimizer': 8}. Best is trial 0 with value: 48.51528888534959.


{'alpha': 0.24052398094282287, 'n_restarts_optimizer': 8, 'kernel': Matern(length_scale=9.42e+04, nu=4.65) + Matern(length_scale=3.96e+04, nu=3.31)}


KeyboardInterrupt: 

In [73]:
#Reconstruct the kernel based on the results from the optuna test.
def reconstruct_kernel(encoding):
    n_kernels = encoding['n_kernels']
    kernels = []
    for i in range(n_kernels):
        kernel_type = encoding[f'kernel_type_{i}']
        length_scale = encoding.get(f'{kernel_type}_{i}_length_scale', None)
        nu = encoding.get(f'{kernel_type}_{i}_nu', None)

        if kernel_type == 'Matern':
            kernel = Matern(length_scale=length_scale, nu=nu)
        elif kernel_type == 'RationalQuadratic':
            alpha = encoding.get(f'{kernel_type}_{i}_alpha', 1.0)
            kernel = RationalQuadratic(length_scale=length_scale, alpha=alpha)
        elif kernel_type == 'WhiteKernel':
            noise_level = encoding.get(f'{kernel_type}_{i}_noise_level', 1.0)
            print(noise_level)
            kernel = WhiteKernel(noise_level=noise_level)
        # Add more conditions for other kernel types if needed

        kernels.append(kernel)

    #noise_level = encoding.get(f'WhiteKernel_{n_kernels}_noise_level', 1)
    #kernel = WhiteKernel(noise_level=noise_level)
    #kernels.append(kernel)

    # Sum the individual kernels to get the final composite kernel
    if n_kernels == 1:
        final_kernel = kernels[0]
    else:
        final_kernel = Sum(kernels[0], kernels[1])
        for j in range(1, n_kernels-1):
            final_kernel = Sum(final_kernel, kernels[j+1])
    return final_kernel

In [74]:
#Check performance with no tuning to ensure performance is improving
sanity_check = GaussianProcessRegressor(kernel=RBF())
sanity_check.fit(X_train.iloc[0:499], y_train.iloc[0:499])
val_pred = sanity_check.predict(X_val)
verif_pred = sanity_check.predict(X_verif)
sanity_verif_error = mean_squared_error(y_verif,verif_pred,squared=False)
sanity_val_error = mean_squared_error(y_val,val_pred,squared=False)
print("SANITY CHECK VALUES:")
print("Verification RMSE:", sanity_verif_error)
print("Validation RMSE:", sanity_val_error)

SANITY CHECK VALUES:
Verification RMSE: 16.005811334029733
Validation RMSE: 16.326713639930887


In [75]:
params = trial.params
print(params)
kernel = reconstruct_kernel(params)
print(kernel)
gp = GaussianProcessRegressor(kernel=kernel,alpha=params['alpha'], n_restarts_optimizer=params['n_restarts_optimizer'])

NameError: name 'trial' is not defined

In [ ]:
gp.fit(X_train.iloc[0:499], y_train.iloc[0:499])

GaussianProcessRegressor(alpha=0.0015488709805414154,
                         kernel=Matern(length_scale=7.62e+04, nu=1.82) + Matern(length_scale=2.41e+04, nu=3.87) + WhiteKernel(noise_level=2.56e+04),
                         n_restarts_optimizer=1)

In [ ]:
val_pred, std_prediction = gp.predict(X_val, return_std=True)
error = mean_squared_error(y_val,val_pred,squared=False)
print("RMSE:", error)
print("Difference from sanity check:", sanity_val_error - error)

RMSE: 4.678907046098204
Difference from sanity check: 11.647806593832684


In [ ]:
val_r = r2_score(y_val,val_pred)

print("val R:")
print(val_r)

val R:
-0.024127588444852188


In [ ]:
if not OUTPUT_TEST:
    raise ValueError("OUTPUT_TEST set to False. If you would like to output final test values set to True and continue running from here")

ValueError: OUTPUT_TEST set to False. If you would like to output final test values set to True and continue running from here

In [ ]:
X_test = pd.read_csv("../data/cleaned/test.csv")
y_test = pd.read_csv("../data/cleaned/test_labels.csv")

In [ ]:
columns = X_test.columns
for col in columns:
    if '[' in col or ']' in col:
        old_name = col
        col = col.replace('[', '(')
        col = col.replace(']', ')')
        
        X_test = X_test.rename(columns={old_name:col})

In [ ]:
test_preds = gp.predict(X_test)
train_preds = gp.predict(X_train)

In [ ]:
#Save test true vals and predictions to csv

pred_data = pd.DataFrame(test_preds)
pred_filepath = '../data/predictions/GP/test_pred_gp.csv'
pred_data.to_csv(pred_filepath, index=False, header=False)
pred_data = pd.DataFrame(y_test)
pred_filepath = '../data/predictions/GP/test_true_gp.csv'
pred_data.to_csv(pred_filepath, index=False, header=False)

#Save train true vals and predictions to csv

pred_data = pd.DataFrame(train_preds)
pred_filepath = '../data/predictions/GP/train_pred_gp.csv'
pred_data.to_csv(pred_filepath, index=False, header=False)
pred_data = pd.DataFrame(y_train)
pred_filepath = '../data/predictions/GP/train_true_gp.csv'
pred_data.to_csv(pred_filepath, index=False, header=False)

In [ ]:
#Save inputs to csv

pred_data = pd.DataFrame(X_train)
pred_filepath = '../data/predictions/GP/train_input_gp.csv'
pred_data.to_csv(pred_filepath, index=False, header=False)
true_data = pd.DataFrame(X_test)
true_filepath = '../data/predictions/GP/test_input_gp.csv'
true_data.to_csv(true_filepath, index=False, header=False)

In [ ]:
#Read in values from csv and calculate RMSE and r values

test_pred_data = np.genfromtxt('../data/predictions/GP/test_pred_gp.csv', delimiter=',', filling_values=np.nan)
test_true_data = np.genfromtxt('../data/predictions/GP/test_true_gp.csv', delimiter=',', filling_values=np.nan)
train_pred_data = np.genfromtxt('../data/predictions/GP/train_pred_gp.csv', delimiter=',', filling_values=np.nan)
train_true_data = np.genfromtxt('../data/predictions/GP/train_true_gp.csv', delimiter=',', filling_values=np.nan)

test_rmse = mean_squared_error(test_true_data,test_pred_data,squared=False)
test_r = r2_score(test_true_data,test_pred_data)

train_rmse = mean_squared_error(train_true_data,train_pred_data,squared=False)
train_r = r2_score(train_true_data,train_pred_data)

print("Train:")
print(train_rmse)
print('Test:')
print(test_rmse)
print(test_r)

Train:
4.6359895675101335
Test:
4.6813758913658905
-0.011374792120455002


In [ ]:
print("percent Error:", mean_absolute_percentage_error(test_true_data, test_pred_data)*100)

percent Error: 74.22485943098808
